In [ ]:
!pip install easyocr
!pip install paddlepaddle-gpu
!pip install paddleocr
!apt-get install tesseract-ocr
!pip install pytesseract
!pip install opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.7 MB/s eta 0:00:00
  

In [ ]:
import cv2
import sys
import re
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import easyocr
import pytesseract
from paddleocr import PaddleOCR
import torch
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import re
# import constants
import os
import requests
import pandas as pd
import multiprocessing
import time
from time import time as timer
from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
import requests
import urllib
from PIL import Image

In [ ]:
!mkdir images

In [ ]:
df = pd.read_csv('/content/train.csv')

In [ ]:
df.shape

(263859, 4)

# RANGE

In [ ]:
df = df[:100000]

In [ ]:

def create_placeholder_image(image_save_path):
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        return

def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except:
            time.sleep(delay)

    create_placeholder_image(image_save_path) #Create a black placeholder image for invalid links/images

def download_images(image_links, download_folder, allow_multiprocessing=True):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(
            download_image, save_folder=download_folder, retries=3, delay=3)

        with multiprocessing.Pool(64) as pool:
            list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
            pool.close()
            pool.join()
    else:
        for image_link in tqdm(image_links, total=len(image_links)):
            download_image(image_link, save_folder=download_folder, retries=3, delay=3)


In [ ]:
download_images(df['image_link'], '/content/images')

100%|██████████| 100000/100000 [12:12<00:00, 136.48it/s]


In [ ]:
def get_file_name(url):
    return re.search(r'/([^/]+\.[a-z]+)$', url).group(1)

In [ ]:
df['filename'] = df.image_link.apply(get_file_name)

In [ ]:
# Check if GPU is available and print status
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Initialize OCR engines
# EasyOCR automatically uses GPU if it's available
# easyocr_reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available())

# PaddleOCR uses GPU if specified
paddle_ocr = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=True)

# Tesseract does not support GPU acceleration natively, so it will run on CPU

# Folder path
base_image_path = Path('/content/images')

# Function to extract text using EasyOCR
# def extract_text_easyocr(filename):
#     image_path = base_image_path / filename
#     try:
#         result = easyocr_reader.readtext(str(image_path), detail=0)
#         return ' '.join(result)
#     except Exception as e:
#         print(f"EasyOCR failed for {filename}: {e}")
#         return None

# Function to extract text using Tesseract OCR (CPU only)
# def extract_text_tesseract(filename):
#     image_path = base_image_path / filename
#     try:
#         image = cv2.imread(str(image_path))
#         gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#         result = pytesseract.image_to_string(gray)
#         return result
#     except Exception as e:
#         print(f"Tesseract failed for {filename}: {e}")
#         return None

# Function to extract text using PaddleOCR (GPU supported)
def extract_text_paddleocr(filename):
    image_path = base_image_path / filename
    try:
        result = paddle_ocr.ocr(str(image_path))
        text = ' '.join([line[1][0] for line in result[0]])
        return text
    except Exception as e:
        print(f"PaddleOCR failed for {filename}: {e}")
        return None

# Apply the OCR functions to the DataFrame
# test['easyocr'] = test['filename'].apply(extract_text_easyocr)
# Uncomment the next line to include Tesseract
# test['tesseract'] = test['filename'].apply(extract_text_tesseract)
df['paddleocr'] = df['filename'].apply(extract_text_paddleocr)

Using device: cuda
download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:00<00:00, 5.13MiB/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:00<00:00, 10.9MiB/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:00<00:00, 3.19MiB/s]

[2024/09/14 10:11:59] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_ch

Streaming output truncated to the last 5000 lines.
[2024/09/14 11:24:22] ppocr DEBUG: cls num  : 12, elapsed : 0.02141094207763672
[2024/09/14 11:24:22] ppocr DEBUG: rec_res num  : 12, elapsed : 0.06714868545532227
[2024/09/14 11:24:22] ppocr DEBUG: dt_boxes num : 8, elapsed : 0.056441545486450195
[2024/09/14 11:24:22] ppocr DEBUG: cls num  : 8, elapsed : 0.02507948875427246
[2024/09/14 11:24:22] ppocr DEBUG: rec_res num  : 8, elapsed : 0.029691219329833984
[2024/09/14 11:24:23] ppocr DEBUG: dt_boxes num : 20, elapsed : 0.0679628849029541
[2024/09/14 11:24:23] ppocr DEBUG: cls num  : 20, elapsed : 0.039101362228393555
[2024/09/14 11:24:23] ppocr DEBUG: rec_res num  : 20, elapsed : 0.07982254028320312
[2024/09/14 11:24:23] ppocr DEBUG: dt_boxes num : 10, elapsed : 0.06284475326538086
[2024/09/14 11:24:23] ppocr DEBUG: cls num  : 10, elapsed : 0.020415782928466797
[2024/09/14 11:24:23] ppocr DEBUG: rec_res num  : 10, elapsed : 0.049779653549194336
[2024/09/14 11:24:23] ppocr DEBUG: dt_bo

In [ ]:
df.to_csv('df1.csv', index=False)